<a href="https://colab.research.google.com/github/Guilhermesena-dev/API_DJE_REQUEST/blob/main/TB02_MINERA%C3%87%C3%83O_DE_DADOS_GUILHERME_CAMPELO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

a) Ler o dataset fakeTelegram.BR_2022.csv

In [1]:
import pandas as pd
import duckdb
from google.colab import drive
import re
drive.mount('/content/drive')


df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fakeTelegram.BR_2022.csv')

df.head()


Mounted at /content/drive


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


b) Remova os trava-zaps.

In [2]:
def trava_zap (text):
      return bool(re.search(r'(.)\1{4,}', str(text))) #detecta uma sequencia de 5 caracteres iguais
remove = df['text_content_anonymous'].apply(trava_zap) #aplica a detecção de trava_zaps
df = df[~remove].reset_index(drop= True) #tira os trava_zaps do df
print("Linhas removidas com sucesso: {remove.sum()}")
df.shape


Linhas removidas com sucesso: {remove.sum()}


(536006, 19)

c) Exportar os dados para um arquivo Parquet.


In [3]:
df.to_parquet('/content/drive/MyDrive/Colab Notebooks/fakeTelegram.parquet.csv', index=False)

d) Exportar os dados para o DuckDB.


In [4]:
con = duckdb.connect(database=':memory:') #conecta o duckdb a memoria
con.register('telegram', df) #cria a tabel telegram
print("Tabela 'telegram' criada em DuckDB na memória.")

Tabela 'telegram' criada em DuckDB na memória.


e) Utlizando o DuckDB recupere:

1. A quantidade de mensagens;


In [5]:
display(con.execute("SELECT COUNT(*) AS total_mensagens FROM telegram").df())


,total_mensagens
0,536006


2. A quantidade de usuários;


In [6]:
display(con.execute("SELECT COUNT(DISTINCT id_member_anonymous) AS total_users FROM TELEGRAM").df())

,total_users
0,14547


3. A quantidade de grupos;


In [7]:
display(con.execute("SELECT COUNT(DISTINCT id_group_anonymous) AS total_goups FROM TELEGRAM").df())

,total_goups
0,178


4. Quantidade de mensagens que possuem apenas texto;


In [8]:
display(con.execute("SELECT COUNT(*) AS apenas_texto from telegram WHERE has_media=FALSE").df())

,apenas_texto
0,212682


5. Quantidade de mensagens contendo mídias;


In [9]:
display(con.execute("SELECT COUNT(*) AS com_midias from telegram WHERE has_media=TRUE").df())

,com_midias
0,323324


6. Quantidade de mensagens por tipo de mídia (jpg, mp4 etc);


In [10]:
qnts_midia = """
SELECT
  media_type,
  COUNT(*) AS quantidade
FROM TELEGRAM
WHERE has_media = TRUE
GROUP BY media_type
ORDER BY quantidade DESC

"""
display(con.execute(qnts_midia).df())

,media_type,quantidade
0,image/jpg,193522
1,url,98843
2,video/mp4,18218
3,application/vnd.android.package-archive,7159
4,application/pdf,2848
...,...,...
62,image/m4a,1
63,audio/m4a,1
64,image/gz,1
65,image/af75130da3b6516eb3c1eb9ab62b5219,1


Itens 7, 8, 9, 10, 11 foram pulados seguindo a orientação do professor.

12. As 30 URLs que mais se repetem (mais compartilhadas);


In [11]:
Q12 = """
SELECT
  media_url AS url,
  COUNT(*)  AS freq
FROM telegram
WHERE media_url IS NOT NULL
  AND media_url <> ''
GROUP BY media_url
ORDER BY freq DESC
LIMIT 31
"""
display(con.execute(Q12).df())

,url,freq
0,t.me/alexeconomia,3718
1,https://youtube.com/c/especulandoosfatosoficial,1600
2,t.me/fimtaproximo,1559
3,t.me/+EWlGMatRZGg3OTlh,1418
4,https://youtu.be/qbTzhB0akt8,1150
5,https://youtu.be/zDuOoyhyN-4,996
6,t.me/+ewZIPdZ42vEyNzJh,772
7,https://t.me/canalselvabrasiloficial,705
8,https://youtu.be/4DHk9KZ01HM,649
9,https://youtu.be/x2uiakywcrI,566


13. Os 30 domínios que mais se repetem (mais compartilhados);


In [12]:
Q13 = """
SELECT
  split_part(media_url, '/', 3) AS domain,
  COUNT(*) AS freq
FROM telegram
WHERE media_url IS NOT NULL
GROUP BY domain
ORDER BY freq DESC
LIMIT 30
"""
display(con.execute(Q13).df())

,domain,freq
0,youtu.be,52129
1,,10549
2,t.me,9646
3,terrabrasilnoticias.com,8161
4,www.youtube.com,8093
5,youtube.com,5908
6,www.instagram.com,5670
7,www.jornaldacidadeonline.com.br,4809
8,twitter.com,4308
9,gazetabrasil.com.br,1774


14. Os 30 usuários mais ativos;


In [13]:
Q14 = """
SELECT
  id_member_anonymous AS usuario,
  COUNT(*) AS total_mensagens
FROM telegram
GROUP BY usuario
ORDER BY total_mensagens DESC
LIMIT 30
"""
display(con.execute(Q14).df())

,usuario,total_mensagens
0,None,309420
1,abe534d581ec6d552243d6955d3c3cd8,12277
2,1665e22b0f564cd46d343f7677014821,5448
3,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4982
4,c743967449a387ad2c1c7e03b2c45b36,2783
5,e7998863ac2a40086657fab4a6b463c9,1896
6,e8fd8fee8c39342d37993775da7756d5,1671
7,2f4be6244ede15b46e8329a2c975be30,1600
8,d49f81df0c75d1d72bee6c5b2d707da0,1564
9,773b9bd5b02a2e96f9d732c29bfcb663,1392


15. Os 30 usuários que mais compartilharam texto;


In [14]:
Q15 = """
SELECT
  id_member_anonymous AS usuario,
  COUNT(*) AS total_texto
FROM telegram
WHERE has_media = FALSE
GROUP BY usuario
ORDER BY total_texto desc
LIMIT 30
"""
display(con.execute(Q15).df())

,usuario,total_texto
0,None,115128
1,abe534d581ec6d552243d6955d3c3cd8,12277
2,c743967449a387ad2c1c7e03b2c45b36,2631
3,1665e22b0f564cd46d343f7677014821,2478
4,d49f81df0c75d1d72bee6c5b2d707da0,1564
5,f233cf8b1d4ede06f32199a7e0081bf5,1011
6,e8fd8fee8c39342d37993775da7756d5,675
7,a378e9743fa3ca297df321cfa0e7cf6a,588
8,7696d5103cdb8ac352d748a1db1126b0,560
9,26b496125c8b6773bce453e14f172430,502


16. Os 30 usuários que mais compartilharam mídias;


In [15]:
Q16 = """
SELECT
  id_member_anonymous AS usuario,
  COUNT(*) AS total_midias
FROM telegram
WHERE has_media = TRUE
GROUP BY usuario
ORDER BY total_midias DESC
LIMIT 30
"""
display(con.execute(Q16).df())

,usuario,total_midias
0,None,194292
1,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4564
2,1665e22b0f564cd46d343f7677014821,2970
3,e7998863ac2a40086657fab4a6b463c9,1495
4,2f4be6244ede15b46e8329a2c975be30,1482
5,4f7d493f0f6222d56e5b19a4f7c336cc,1279
6,c052c859b42c5a1923c22f5a201de746,1272
7,773b9bd5b02a2e96f9d732c29bfcb663,1156
8,56b8359fd127312651b80b8ed8030085,1067
9,e8fd8fee8c39342d37993775da7756d5,996


17. As 30 mensagens mais compartilhadas;


In [16]:
Q17 = """
SELECT
  text_content_anonymous AS mensagem,
  COUNT(*) AS freq

FROM telegram
GROUP BY mensagem
ORDER BY freq DESC
LIMIT 30
"""
display(con.execute(Q17).df())



,mensagem,freq
0,None,113385
1,This community was blocked in Brazil following...,17422
2,Rough_sex🙈,1134
3,Anal sex🙈,1118
4,سکس مردان ازبک با زن انگلیسی با این vpn از سای...,1019
5,https://youtu.be/qbTzhB0akt8,758
6,https://youtu.be/zDuOoyhyN-4,712
7,فیلم سوپر با زیرنویس فارسی ببین😍😍\nبا این فیلت...,632
8,Rough😱,570
9,Foto de Nélia Barros,548


18. As 30 mensagens mais compartilhadas em grupos diferentes;


In [17]:
Q18 = """
SELECT
  text_content_anonymous AS mensagem,
  COUNT(DISTINCT id_group_anonymous) AS total_grupos

FROM telegram
GROUP BY mensagem
ORDER BY total_grupos DESC
LIMIT 30
"""
display(con.execute(Q18).df())

,mensagem,total_grupos
0,None,161
1,This community was blocked in Brazil following...,59
2,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,36
3,Hoje às 18 horas no canal Inteligência Ltda do...,35
4,[USER],32
5,https://youtu.be/zDuOoyhyN-4,31
6,https://youtu.be/qbTzhB0akt8,30
7,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,29
8,"DE NADA IRÁ ADIANTAR FISCALIZAR AS URNAS, SE N...",29
9,Ajudem a subir a #JanonesAmigoDePedofilo\n\nPo...,29


19. Mensagens idênticas compartilhadas pelo mesmo usuário (e suas
quantidades);

In [18]:
Q19 = """
SELECT
    id_member_anonymous AS usuario,
    text_content_anonymous AS mensagem,
    COUNT(*) AS freq
FROM telegram
GROUP BY usuario, mensagem
HAVING freq > 1
ORDER BY freq DESC
"""

display(con.execute(Q19).df())


,usuario,mensagem,freq
0,None,None,66583
1,None,This community was blocked in Brazil following...,9648
2,2f4be6244ede15b46e8329a2c975be30,None,1320
3,1ac091b8ed5c4e42383f1b4ff4cc9b2d,None,1038
4,f233cf8b1d4ede06f32199a7e0081bf5,This community was blocked in Brazil following...,838
...,...,...,...
46976,None,Bolsonaro cresceu nas urnas do exterior; Veja ...,2
46977,a615e63a447e27c0b0291978940f14fc,https://youtu.be/NsLlT0UGe60,2
46978,c052c859b42c5a1923c22f5a201de746,"🟢""E conhecereis a verdade, e a verdade vos lib...",2
46979,72098fe1ff89793a882789bbc8d82abc,https://youtu.be/5qCk5znO-v4,2


20. Mensagens idênticas compartilhadas pelo mesmo usuário em grupos
distintos (e suas quantidades);

In [19]:
Q20 = """
SELECT
    id_member_anonymous AS usuario,
    text_content_anonymous AS mensagem,
    COUNT(DISTINCT id_group_anonymous) AS qtd_grupos
FROM telegram
GROUP BY usuario, mensagem
HAVING qtd_grupos > 1
ORDER BY qtd_grupos DESC
"""
display(con.execute(Q20).df())

,usuario,mensagem,qtd_grupos
0,None,None,144
1,None,This community was blocked in Brazil following...,53
2,None,Hoje às 18 horas no canal Inteligência Ltda do...,32
3,None,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,32
4,None,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,28
...,...,...,...
33691,None,ATENÇÃO!! O relatório das FA aponta falhas gra...,2
33692,89f235ebc8aea77671b7428c0d2a40d8,https://www.instagram.com/reel/Cj8cmUdsf65/?ig...,2
33693,858ca55953c9c38bcd45e23d302658ae,Enviando por email Relatorio_EFASEV.pdf,2
33694,89f235ebc8aea77671b7428c0d2a40d8,https://twitter.com/jean_c_d_/status/159052110...,2


21. Os 30 unigramas, bigramas e trigramas mais compartilhados;


In [20]:
Q21 = """
WITH tokens AS (
  SELECT
    LOWER(t.token) AS unigram
  FROM telegram
    CROSS JOIN UNNEST(string_split(text_content_anonymous, ' ')) AS t(token)
  WHERE text_content_anonymous IS NOT NULL
)
SELECT
  unigram,
  COUNT(*) AS freq
FROM tokens
GROUP BY unigram
ORDER BY freq DESC
LIMIT 30
"""
display(con.execute(Q21).df())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,unigram,freq
0,de,401772
1,o,305731
2,a,298746
3,e,297276
4,que,260540
5,do,175915
6,para,134644
7,não,130048
8,é,127693
9,no,112247


22. As 30 mensagens mais positivas (distintas);


In [21]:
Q22 = """
SELECT DISTINCT
  text_content_anonymous AS mensagem,
  score_sentiment
FROM telegram
ORDER BY score_sentiment DESC
LIMIT 30
"""

display(con.execute(Q22).df())

,mensagem,score_sentiment
0,"Shemoneh Esreh – Amidá – A Grande Oração\n\n ,...",0.9992
1,"MENTIROSA!!!\n\n\nPensa direita, não é correto...",0.9990
2,"Pensa direita, não é correto compartilhar um v...",0.9990
3,"בונה ירושלים, Bo'ne Yetrushalayim - Benção da ...",0.9989
4,Os Sete Samurais \n \nQuando falamos sobre For...,0.9988
5,MANIFESTO À NAÇÃO BRASILEIRA\n\nEM DEFESA DO B...,0.9987
6,BOM DIA A TODOS. ENTENDO QUE ESTAMOS VIVENDO D...,0.9986
7,"Olá ,\n\nUM AMIGO ME PERGUNTOU EM QUEM EU VOTE...",0.9986
8,\nA Viva Esperança\n\nDepois das saudações in...,0.9985
9,Uma forma bem legal\nPara quem ainda não sabe ...,0.9985


23. As 30 mensagens mais negativas (distintas);


In [22]:
Q23 = """
SELECT DISTINCT
  text_content_anonymous AS mensagem,
  score_sentiment
FROM telegram
ORDER BY score_sentiment ASC
LIMIT 30
"""

display(con.execute(Q23).df())

,mensagem,score_sentiment
0,CAPITÃOZINHO DE MERDA 💩\nCAPITÃOZINHO DE MERDA...,-1.0000
1,fraude fraude fraude fraude fraude fraude frau...,-0.9999
2,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
3,"E nós aqui nos sacrificando, aguentando sol, c...",-0.9999
4,ME CÚ NO PAU DO BOLSONARO \nME CÚ NO PAU DO BO...,-0.9999
5,"Gente, a Zambelli nos abandonou! Traiu a nossa...",-0.9999
6,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
7,ai ta na rua da um joinha 👍Quem ai ta na rua d...,-0.9999
8,Sobre a morte como política global e a necessi...,-0.9998
9,Quem ai ta na rua da um joinha 👍Quem ai ta na ...,-0.9998


24. O usuário mais otimista;


In [23]:
Q24 = """
SELECT
  id_member_anonymous AS usuario,
  AVG(score_sentiment) AS media_sentimento

FROM telegram
GROUP BY usuario
ORDER BY media_sentimento DESC
LIMIT 1
"""
display(con.execute(Q24).df())

,usuario,media_sentimento
0,016723c9eb650754d3dfe9b3ba69e5c2,0.9953


25. O usuário mais pessimista;


In [24]:
Q25 = """
SELECT
  id_member_anonymous AS usuario,
  AVG(score_sentiment) AS media_sentimento

FROM telegram
GROUP BY usuario
ORDER BY media_sentimento ASC
LIMIT 1
"""
display(con.execute(Q25).df())

,usuario,media_sentimento
0,0716fd79b67254844bc02de12d9e084b,-0.9999


26. As 30 maiores mensagens;


In [25]:
Q26 = """
SELECT
  TEXT_CONTENT_ANONYMOUS AS mensagem,
  LENGTH(TEXT_CONTENT_ANONYMOUS) AS tamanho_mensagem
FROM TELEGRAM
ORDER BY tamanho_mensagem DESC
LIMIT 30
"""
display(con.execute(Q26).df())

,mensagem,tamanho_mensagem
0,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
1,COMO O SOCIALISMO DESTRUIU A VENEZUELA EM POUC...,4096
2,"1: Sansão foi a Gaza, e viu ali uma prostituta...",4096
3,[Encaminhado de FYI-Brasil (Sú)]\n🚨ELEIÇÕES DE...,4096
4,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
5,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
6,"1: Aconteceu também naqueles dias, quando não ...",4096
7,1: Naqueles dias não havia melech em Yisra-EL;...,4096
8,"1: Aconteceu também naqueles dias, quando não ...",4096
9,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096


27. As 30 menores mensagens;


In [26]:
Q27 = """
SELECT
  TEXT_CONTENT_ANONYMOUS AS mensagem,
  LENGTH(TEXT_CONTENT_ANONYMOUS) AS tamanho_mensagem
FROM TELEGRAM
ORDER BY tamanho_mensagem ASC
LIMIT 30
"""
display(con.execute(Q27).df())

,mensagem,tamanho_mensagem
0,🌩,1
1,💣,1
2,🤭,1
3,/,1
4,🌹,1
5,🤣,1
6,🤔,1
7,😂,1
8,😢,1
9,😂,1


28. O dia em que foi publicado a maior quantidade de mensagens;


In [30]:
Q28 = """
SELECT
  CAST(date_message AS DATE) AS dia,
  COUNT(*) AS total
FROM telegram
GROUP BY dia
ORDER BY total DESC
LIMIT 1
"""
display(con.execute(Q28).df())

,dia,total
0,2022-10-02,24469


29. As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”;


In [31]:
Q29 = """
SELECT *
FROM telegram
WHERE text_content_anonymous LIKE '%FACÇÃO%' AND text_content_anonymous LIKE '%CRIMINOSA%'
"""
display(con.execute(Q29).df())


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 18:58:08,None,3125db581fc8ba28fa9aa2a4e1ac3a0a,None,None,https://gettr.com/post/p1t8lax7bb8,False,True,False,#lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMIN...,5,2022-10-05 18:58:12.15472,0.0000,NaN,22872,Texto,telegram,None,None
1,2022-10-12 07:40:03,None,5b10d7739171149be6d9961e3350c071,None,None,None,False,False,False,PT NÃO É PARTIDO POLÍTICO É FACÇÃO CRIMINOSA ....,5,2022-10-12 07:40:10.047528,-0.7761,NaN,1119975,Texto,telegram,None,None
2,2022-10-13 00:27:03,3f1819e6b98ea987f466b2ffd54b9c5e,5b10d7739171149be6d9961e3350c071,None,None,None,False,False,False,🚨 LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 00:27:04.70799,0.0000,0.607883,1123161,Texto,telegram,None,None
3,2022-10-13 00:27:48,3f1819e6b98ea987f466b2ffd54b9c5e,5b10d7739171149be6d9961e3350c071,None,None,None,False,False,False,🚨 LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 00:28:04.037146,0.0000,0.607883,1123168,Texto,telegram,None,None
4,2022-10-13 00:30:32,3f1819e6b98ea987f466b2ffd54b9c5e,5b10d7739171149be6d9961e3350c071,None,None,None,False,False,False,🚨 LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 00:30:38.29219,0.0000,0.607883,1123190,Texto,telegram,None,None
5,2022-10-13 01:05:32,None,857cd5311da1bdc15eb9e6918a47c6c6,03daf8bf484996bec4ce61d1f79e1ad3.jpg,image/jpg,None,True,False,False,- LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 01:05:36.716217,-0.5848,0.728353,913350,Imagem,telegram,None,03daf8bf484996bec4ce61d1f79e1ad3
6,2022-10-13 01:16:41,None,1cf9e56465ddb9766a23a7da49fecb48,3adb398c43bd637d9cde192d7d5c3d38.jpg,image/jpg,None,True,False,False,- LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 01:16:43.660339,-0.5848,0.743789,9923,Imagem,telegram,None,3adb398c43bd637d9cde192d7d5c3d38
7,2022-10-13 12:55:40,78751128cc688dd52ba9ab7a7cc87253,b11f2df64ac19aad47a50accf32052d6,None,None,None,False,False,False,🚨 LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 12:55:43.189611,0.0000,0.607883,162586,Texto,telegram,None,None
8,2022-10-13 01:19:42,fbbbfb737acb18812f7de2400841abc3,857cd5311da1bdc15eb9e6918a47c6c6,3adb398c43bd637d9cde192d7d5c3d38.jpg,image/jpg,None,True,False,False,- LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 01:20:10.261728,-0.8356,0.956106,913398,Imagem,telegram,None,3adb398c43bd637d9cde192d7d5c3d38
9,2022-10-13 01:19:43,fbbbfb737acb18812f7de2400841abc3,d9c080b9db8cd9d0e88e870ce782c01a,3adb398c43bd637d9cde192d7d5c3d38.jpg,image/jpg,None,True,False,False,- LULA USA BONE COM SIGLA CPX NO COMPLEXO DO A...,5,2022-10-13 01:20:02.504407,-0.8356,0.956106,318394,Imagem,telegram,None,3adb398c43bd637d9cde192d7d5c3d38


30. As mensagens que possuem a palavra “SEGURANÇA”.


In [32]:
Q30 = """
SELECT *
FROM telegram
WHERE text_content_anonymous LIKE '%SEGURANÇA%'
"""
display(con.execute(Q30).df())

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 17:22:23,627bcf3e89d46fc566db14354992b76b,8cd0567564cb9ee47545dbccaec3f024,06c4942d698b72cb060c40c36bed830c.jpg,image/jpg,None,True,False,False,Espalhe!! Divulgue!! EXIJA!!\nNão entre nesse ...,5,2022-10-05 17:22:28.126598,-0.5526,0.614709,10053,Imagem,telegram,None,06c4942d698b72cb060c40c36bed830c
1,2022-10-05 19:09:29,bbfb7173f814789890deebb5fa7d5aa3,30cf2ff94ba278b980ef403a077af417,None,None,None,False,False,False,ATENÇÃO A TODAS E SOMENTE MULHERES DESTE GRUPO...,5,2022-10-05 19:09:35.543291,0.1196,0.044148,7852,Texto,telegram,None,None
2,2022-10-05 20:54:29,None,4b07f6441df07b2f5da8c349eb29e340,c34397d9ac236c7249b770ceb80b9e31.jpg,image/jpg,None,True,False,False,Espalhe!! Divulgue!! EXIJA!!\nNão entre nesse ...,5,2022-10-05 20:55:15.649236,-0.5526,0.614709,3519,Imagem,telegram,None,c34397d9ac236c7249b770ceb80b9e31
3,2022-10-05 21:26:22,None,a8743228062bf30f47ccba061079efb4,None,None,None,False,False,False,Bolsonaro ou Lula?\nO que cada qual já fez e p...,5,2022-10-05 21:26:23.112854,-0.9661,0.996412,105445,Texto,telegram,None,None
4,2022-10-06 04:18:20,fb4a5f8c3319aa373d868e69589481f9,5b10d7739171149be6d9961e3350c071,2feb133e1f34617216a5e62bbf87e308.jpg,url,https://ift.tt/klH2aVu,True,True,False,*VIDEO: FUNCIONARIO DA ONU ADMITE QUE ELES CON...,5,2022-10-06 04:18:27.238603,-0.9897,0.999983,1095148,Url,telegram,None,2feb133e1f34617216a5e62bbf87e308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,2022-10-24 06:59:36,d6e28ca3a6812c25e19ace22e18bcd67,5b10d7739171149be6d9961e3350c071,2c6fa696e7b22bb84537e29b89ea87db.jpg,url,https://youtu.be/ppoa5oA_IqE,True,True,False,"PT= CAOS, TROCA DE TIROS, ARBITRARIEDADES, PRI...",5,2022-10-24 07:00:12.863724,-0.8707,0.996626,1176374,Url,telegram,None,2c6fa696e7b22bb84537e29b89ea87db
303,2022-10-31 10:50:33,bedad20110f83a827f53735338b6f4bb,f0944ed1832d8c1f1213d6123746f111,ff215de5c06143eaba9ece7258c8f036.mp4,video/mp4,None,True,False,False,PRA VC QUE É CONTRA AS ARMAS E O BOLSONARO... ...,5,2022-10-31 10:50:37.800762,0.7101,0.602613,7525,Video,telegram,VID-20221031-WA0002.mp4,ff215de5c06143eaba9ece7258c8f036
304,2022-10-31 10:50:34,bedad20110f83a827f53735338b6f4bb,b8a8737812c7fd7d3e0bdbb65ef6306f,ff215de5c06143eaba9ece7258c8f036.mp4,video/mp4,None,True,False,False,PRA VC QUE É CONTRA AS ARMAS E O BOLSONARO... ...,5,2022-10-31 10:50:38.515344,0.7101,0.602613,52809,Video,telegram,VID-20221031-WA0002.mp4,ff215de5c06143eaba9ece7258c8f036
305,2022-10-31 11:26:30,7eeb43c85fe3d87c2de509828d213d80,d9c080b9db8cd9d0e88e870ce782c01a,ff215de5c06143eaba9ece7258c8f036.mp4,video/mp4,None,True,False,False,PRA VC QUE É CONTRA AS ARMAS E O BOLSONARO... ...,5,2022-10-31 11:27:22.495835,0.7101,0.602613,328138,Video,telegram,VID-20221031-WA0002.mp4,ff215de5c06143eaba9ece7258c8f036
